### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pickle

In [2]:
data=pd.read_csv('data/Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# OneHotEncode 'Gender'
onehot_encoder_gender = OneHotEncoder()
gender_encoder = onehot_encoder_gender.fit_transform(data[['Gender']]).toarray()
gender_encoded_df=pd.DataFrame(gender_encoder,columns=onehot_encoder_gender.get_feature_names_out(['Gender']))

# OneHotEncode 'Geography'
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# Concatenate the encoded columns back to the dataframe and drop the original columns
data = pd.concat([data, gender_encoded_df, geo_encoded_df], axis=1).drop(['Gender', 'Geography'], axis=1)

## Save the encoders and sscaler
with open('artifacts/onehot_encoder_gender.pkl','wb') as file:
    pickle.dump(onehot_encoder_gender,file)

with open('artifacts/onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

with open('artifacts/scaler_churn.pkl','wb') as file:
    pickle.dump(scaler,file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [4]:
## Create a Keras classifier
model=KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)

In [5]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits



d:\Work & Study\Study & Projects\Customer Churn Prediction\venv_cus_churn\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)



Epoch 1/50


250/250 [==============================] - 1s 938us/step - loss: 0.5140 - accuracy: 0.7926
Epoch 2/50
250/250 [==============================] - 0s 897us/step - loss: 0.4367 - accuracy: 0.8080
Epoch 3/50
250/250 [==============================] - 0s 886us/step - loss: 0.4095 - accuracy: 0.8266
Epoch 4/50
250/250 [==============================] - 0s 892us/step - loss: 0.3857 - accuracy: 0.8420
Epoch 5/50
250/250 [==============================] - 0s 871us/step - loss: 0.3683 - accuracy: 0.8516
Epoch 6/50
250/250 [==============================] - 0s 884us/step - loss: 0.3588 - accuracy: 0.8545
Epoch 7/50
250/250 [==============================] - 0s 875us/step - loss: 0.3516 - accuracy: 0.8569
Epoch 8/50
250/250 [==============================] - 0s 884us/step - loss: 0.3474 - accuracy: 0.8579
Epoch 9/50
250/250 [==============================] - 0s 872us/step - loss: 0.3440 - accuracy: 0.8584
Epoch 10/50
250/250 [==============================] - 0s 874us/step - loss: 0.